In [55]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import MaxPooling2D,Conv2D,Dropout,Flatten,BatchNormalization


In [6]:
data_path = 'C:\\Users\\sanja\\PycharmProjects\\Music_Genre\\data_10.json'

str

In [104]:
def load_data(data_path):
  

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    print("Data succesfully loaded!")
    return X, y

In [105]:
X, y = load_data(data_path)

Data succesfully loaded!


In [106]:
print(X.shape)
print(y.shape)

(9986, 130, 13)
(9986,)


In [107]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=0.2)

In [108]:
print(X_train.shape)
print(X_test.shape)

(5991, 130, 13)
(2497, 130, 13)


In [99]:
X_train = X_train[...,np.newaxis]
X_test = X_test[...,np.newaxis]
X_validation = X_validation[...,np.newaxis]
#X_train = X_train.reshape(5991,130,13,1)

In [100]:
X_train.shape

(5991, 130, 13, 1)

In [78]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [79]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(Conv2D(32,(3,3),input_shape=input_shape,activation = 'relu'))
    model.add(MaxPooling2D((3,3),padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv2D(32,(3,3),activation = 'relu'))
    model.add(MaxPooling2D((3,3),padding='same'))
    model.add(BatchNormalization())
    
    
#     model.add(Conv2D(5,(3,3),activation = 'relu'))
#     model.add(MaxPooling2D((3,3),padding='same'))
#     model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(10, activation='softmax'))
    
    model.compile(Adam(lr = 0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
    return model

In [80]:
model = create_model(input_shape)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 128, 11, 32)       320       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 43, 4, 32)         0         
_________________________________________________________________
batch_normalization_v1_13 (B (None, 43, 4, 32)         128       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 41, 2, 32)         9248      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 14, 1, 32)         0         
_________________________________________________________________
batch_normalization_v1_14 (B (None, 14, 1, 32)         128       
_________________________________________________________________
flatten_3 (Flatten)          (None, 448)               0         
__________

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [152]:
X_to_predict = X_test[101]
y_to_predict = y_test[101]

In [153]:
def predict(model, X, y):

    X = X[np.newaxis, ...] 
    X = X[...,np.newaxis] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

In [154]:
predict(model,X_to_predict,y_to_predict)

Target: 1, Predicted label: [1]
